In [1]:
from vlhf.hugging_face import HuggingFace
from vlhf.visual_layer import VisualLayer
import os
from dotenv import load_dotenv

# Authentication
load_dotenv()

HF_TOKEN = os.getenv("HF_TOKEN")
VL_USER_ID = os.getenv("VL_USER_ID")
VL_ENV = os.getenv("VL_ENV")

hf = HuggingFace(HF_TOKEN)
vl = VisualLayer(VL_USER_ID, VL_ENV)

2024-08-07 19:38:03.048 | INFO     | vlhf.hugging_face:__init__:88 - Hugging Face session created
2024-08-07 19:38:03.049 | INFO     | vlhf.visual_layer:__init__:20 - Visual Layer session created


In [ ]:
# hf.download_dataset("keremberke/german-traffic-sign-detection", name="full", image_key="image", bbox_key="objects", bbox_label_names=['animals', 'construction', 'cycles crossing', 'danger', 'no entry', 'pedestrian crossing', 'school crossing', 'snow', 'stop', 'bend', 'bend left', 'bend right', 'give way', 'go left', 'go left or straight', 'go right', 'go right or straight', 'go straight', 'keep left', 'keep right', 'no overtaking', 'no overtaking -trucks-', 'no traffic both ways', 'no trucks', 'priority at next intersection', 'priority road', 'restriction ends', 'restriction ends -overtaking -trucks--', 'restriction ends -overtaking-', 'restriction ends 80', 'road narrows', 'roundabout', 'slippery road', 'speed limit 100', 'speed limit 120', 'speed limit 20', 'speed limit 30', 'speed limit 50', 'speed limit 60', 'speed limit 70', 'speed limit 80', 'traffic signal', 'uneven road'])

In [ ]:
# hf.download_dataset("keremberke/csgo-object-detection", name="full", 
#                     image_key="image", 
#                     bbox_key="objects", 
#                     bbox_label_names=['ct', 'cthead', 't', 'thead']
# )

In [ ]:
# hf.download_dataset("rishitdagli/cppe-5", 
#                     image_key="image", 
#                     bbox_key="objects", 
#                     bbox_label_names=["coverall", "face_shield", "gloves", "goggles", "mask"])

In [ ]:
# hf.download_dataset("Francesco/apples-fvpl5", 
#                     image_key="image", 
#                     bbox_key="objects", 
#                     bbox_label_names=["apple", "damaged_apple"]
# )

In [2]:
hf.download_dataset("Francesco/weed-crop-aerial", 
                    image_key="image", 
                    bbox_key="objects", 
                    bbox_label_names=["crop", "weed"]
)

2024-08-07 19:38:08.944 | INFO     | vlhf.hugging_face:download_dataset:134 - Downloading dataset Francesco/weed-crop-aerial and saving to local path Francesco/weed-crop-aerial


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

2024-08-07 19:38:52.802 | INFO     | vlhf.hugging_face:download_dataset:140 - Adding image filename to dataset


Map:   0%|          | 0/1176 [00:00<?, ? examples/s]

2024-08-07 19:38:53.313 | INFO     | vlhf.hugging_face:download_dataset:145 - Adding bbox label name to dataset
2024-08-07 19:38:54.873 | INFO     | vlhf.hugging_face:is_one_indexed:34 - Minimum category label: 1
2024-08-07 19:38:54.873 | INFO     | vlhf.hugging_face:is_one_indexed:35 - Maximum category label: 2
2024-08-07 19:38:54.874 | INFO     | vlhf.hugging_face:is_one_indexed:36 - Number of unique categories: 2
2024-08-07 19:38:54.874 | INFO     | vlhf.hugging_face:is_one_indexed:42 - Dataset appears to be 1-indexed


Map:   0%|          | 0/1176 [00:00<?, ? examples/s]

Saving images:   0%|          | 0/1176 [00:00<?, ?it/s]

In [ ]:
hf.dataset[0]

In [3]:
hf.to_vl(vl_session=vl)

2024-08-07 19:38:58.516 | INFO     | vlhf.hugging_face:to_vl:206 - Preparing upload to Visual Layer
2024-08-07 19:39:00.339 | INFO     | vlhf.visual_layer:create_dataset:24 - Creating dataset: weed-crop-aerial


Dataset url: https://app.visual-layer.com/dataset/a42f9fe0-54b1-11ef-b422-ea2abd376502


2024-08-07 19:39:33.801 | INFO     | vlhf.visual_layer:create_dataset:26 - Dataset weed-crop-aerial successfully created in Visual Layer!


In [ ]:
! tar -xvf /home/dnth/Desktop/vl-hf-workflow/notebooks/saved_images/keremberke/csgo-object-detection.tar -C /home/dnth/Desktop/vl-hf-workflow/notebooks/saved_images/keremberke

In [ ]:
import pandas as pd

df = pd.read_parquet("saved_images/keremberke/object_annotations.parquet")

In [ ]:
df

In [ ]:
# plot the bounding boxes from the dataframe
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

def plot_bounding_boxes_from_df(df):
    for filename, group in df.groupby('filename'):
        image = Image.open(f"saved_images/keremberke/{filename}")
        draw = ImageDraw.Draw(image)
        
        for i, row in group.iterrows():
            x, y, w, h = row['col_x'], row['row_y'], row['width'], row['height']
            label = row['label']
            draw.rectangle([x, y, x+w, y+h], outline="red", width=3)
            left, top, right, bottom = draw.textbbox((x, y-10), str(label))
            draw.rectangle((left-5, top-5, right+5, bottom+5), fill="red")
            draw.text((x, y-10), str(label), fill="white")

        
        plt.figure(figsize=(12, 8))
        plt.imshow(image)
        plt.axis('off')
        plt.title(f"Image: {filename}")
        plt.show()

In [ ]:
plot_bounding_boxes_from_df(df.sample(5))